In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asyncio
from lib.clue import Clue, DeductiveSolver
from lib.rl import (
    Completion,
    CompletionSampler,
    Episode,
    EpisodeBuffer,
    EpisodeSampler,
    EpisodeSamplerRouter,
)
from lib.tokenizer import Tokenizer
import re
from typing import Coroutine

/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


In [3]:
from lib.vllm import start_vllm_server, vllm_server_metrics

model = "NousResearch/Hermes-2-Theta-Llama-3-8B"

shutdown_server, client = await start_vllm_server(
    disable_log_requests=True,
    model=model,
)

INFO 11-07 23:14:09 api_server.py:528] vLLM API server version dev
INFO 11-07 23:14:09 api_server.py:529] args: Namespace(host=None, port=8001, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key='vllm-95f9a82b4ab473334457340fdc8b00d7', lora_modules=None, prompt_adapters=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=False, disable_frontend_multiprocessing=False, enable_auto_tool_choice=False, tool_call_parser=None, tool_parser_plugin='', model='NousResearch/Hermes-2-Theta-Llama-3-8B', tokenizer=None, skip_tokenizer_init=False, revision=None, code_revision=None, tokenizer_revision=None, tokenizer_mode='auto', trust_remote_code=False, download_dir=None, load_format='auto', config_format='auto', dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, max_model_len=Non

/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.54it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.29it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.69it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.55it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.53it/s]



INFO 11-07 23:14:37 api_server.py:232] vLLM to use /tmp/tmp_6vicj8y as PROMETHEUS_MULTIPROC_DIR
WARNING 11-07 23:14:37 serving_embedding.py:199] embedding_mode is False. Embedding API will not work.
INFO 11-07 23:14:37 launcher.py:19] Available routes are:
INFO 11-07 23:14:37 launcher.py:27] Route: /openapi.json, Methods: HEAD, GET
INFO 11-07 23:14:37 launcher.py:27] Route: /docs, Methods: HEAD, GET
INFO 11-07 23:14:37 launcher.py:27] Route: /docs/oauth2-redirect, Methods: HEAD, GET
INFO 11-07 23:14:37 launcher.py:27] Route: /redoc, Methods: HEAD, GET
INFO 11-07 23:14:37 launcher.py:27] Route: /health, Methods: GET
INFO 11-07 23:14:37 launcher.py:27] Route: /tokenize, Methods: POST
INFO 11-07 23:14:37 launcher.py:27] Route: /detokenize, Methods: POST
INFO 11-07 23:14:37 launcher.py:27] Route: /v1/models, Methods: GET
INFO 11-07 23:14:37 launcher.py:27] Route: /version, Methods: GET
INFO 11-07 23:14:37 launcher.py:27] Route: /v1/chat/completions, Methods: POST
INFO 11-07 23:14:37 launch

In [133]:
shutdown_server()

True

In [4]:
completion_sampler = CompletionSampler(
    client,
    model=model,
)

In [5]:
def sample_random_episode() -> Episode:
    game = Clue(
        num_players=3,
        elements={
            "suspect": Clue.suspects[:3],
            "weapon": Clue.weapons[:3],
            "room": Clue.rooms[:3],
            # "motive": Clue.motives[:6],
            # "time": Clue.get_times("21:00", "03:00", "1h"),
        },
    )
    game.play(
        deductive_solver=DeductiveSolver(
            # note_cards_in_hand=False,
            # note_responses_to_suggestions=False,
            # note_cards_that_players_do_not_have=False,
            # check_unique_card_placement_constraints=False,
            # check_player_hand_size_constraints=False,
            check_solution_has_one_and_only_one_card_per_element=False,
            check_one_of_constraints=False,
            check_inverse_one_of_constraints=False,
            merge_and_check_disjoint_inverse_one_of_constraints=False,
            exhaustively_test_possible_assignments=False,
        ),
        cp_solver_max_solve_time_per_turn=0.05,
        check_cp_solver_grid=False,
        check_if_deductive_solver_and_cp_solver_grids_match=False,
        print_playthrough=False,
    )
    prompt = game.get_prompt()
    follow_up = "Fill out your answer like this:\n" + "\n".join(
        f"{element.capitalize()}: <#{element.upper()}#>" for element in game.elements
    )

    async def reward_completion(completion: Completion) -> None:
        chat_completion = await client.chat.completions.create(
            messages=completion.all_message_params()
            + [
                {"role": "user", "content": follow_up},
            ],
            model=model,
            temperature=0.0,
        )
        answer = chat_completion.choices[0].message.content
        assert answer
        completion.reward = sum(
            [
                bool(
                    re.search(
                        f"{element}: {solution}",
                        answer,
                        re.IGNORECASE,
                    )
                )
                for element, solution in game.solution.items()
            ]
        ) / len(game.solution)

    async def on_sample(completions: list[Completion]) -> None:
        await asyncio.gather(
            *[reward_completion(completion) for completion in completions]
        )
        for completion in completions:
            completion.commit()

    return Episode(
        messages=[{"role": "user", "content": prompt}],
        on_sample=on_sample,
    )

In [6]:
tokenizer = Tokenizer(model)

INFO 11-07 23:14:39 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='NousResearch/Hermes-2-Theta-Llama-3-8B', speculative_config=None, tokenizer='NousResearch/Hermes-2-Theta-Llama-3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=NousResearch/Hermes-2-Theta-Llama-3-8B, use_v2_block_manager=True, num_scheduler_steps=1, chunked_

In [7]:
episode_buffer = EpisodeBuffer(
    episode_sampler_router=EpisodeSamplerRouter(
        EpisodeSampler(sample_random_episode),
        exploitation_factor=1.0,
        min_random_episode_sample_probability_half_life=40,
    ),
    completion_sampler=completion_sampler,
    tokenizer=tokenizer,
    branch_factor=2,
    split_method="count",
    size=64,
)

In [92]:
episode_buffer.start_buffering()

True

<class 'pydantic_core._pydantic_core.ValidationError'> 

In [115]:
running, pending = vllm_server_metrics()
print(f"Running/Pending vLLM Requests: {running}/{pending}")
print("Number of Episodes:", len(episode_buffer.episodes))
print(
    "Pending Tasks:",
    sum(task._state == "PENDING" for task in episode_buffer.tasks.values()),
)
print(
    "Sampled Completions:",
    sum(episode.num_samples() for episode in episode_buffer.episodes),
)

Running/Pending vLLM Requests: 47/0
Number of Episodes: 64
Pending Tasks: 37
Sampled Completions: 2006


In [85]:
for episode in episode_buffer.episodes:
    episode.completion._cached_value = None
    for descendent in episode.completion.descendants():
        descendent._cached_value = None

In [89]:
# %%timeit
for episode in episode_buffer.episodes:
    episode.completion._cached_value = None
    for descendent in episode.completion.descendants():
        descendent._cached_value = None
best_leaves = [
    episode.best_leaf(tokenizer) for episode in episode_buffer.episodes
]

In [88]:
scores = [leaf.all_abs_advantage_per_token(tokenizer, cache=True) for leaf in best_leaves]

In [66]:
print(
    "Average Absolute Advantage Per Token:",
    sum(
        len(episode.best_leaf(tokenizer).children)#.all_abs_advantage_per_token(tokenizer)
        for episode in episode_buffer.episodes
    )
    / len(episode_buffer.episodes),
)

Average Absolute Advantage Per Token: 0.0


In [33]:
sum(1 for _ in episode_buffer.episodes[0].completion.descendants())

7

In [20]:
leaf = episode_buffer.episodes[0].best_leaf(tokenizer)
leaf.all_abs_advantage() / leaf.all_token_count(tokenizer)

0.00029788501638367595

In [180]:
print(completion.all_message_params()[-1]["content"])

Let's analyze the cards shown to each player:

1. Blake shows the Lounge to Robert.
2. Robert shows the Knife to Blake.
3. Joel shows the Knife to Blake.
4. Blake shows the Candlestick to Robert.
5. Joel shows a card to Blake (must be Mrs. White or the Lead Pipe).
6. Blake shows a card to Joel (must be Mr. Green or the Dining Room).
7. Joel shows the Dining Room to Robert.
8. Joel shows a card to Blake (must be the Lead Pipe).
9. Blake shows a card to Robert (must be Mr. Green).

From the information above, we can determine the following:

- Robert has Miss Scarlet and the Knife.
- Joel has Mrs. White and the Dining Room.
- Blake has Mr. Green and the Candlestick.

Since Blake showed the Candlestick to Robert, and Robert has the Knife, we know that the Candlestick is not the Knife. Therefore, the Candlestick must be the weapon in the Lounge.

Similarly, since Blake showed a card to Joel (the Lead Pipe), and Joel has the Dining Room, we know that the Lead Pipe is not the Dining Room. Th

In [141]:
from dataclasses import dataclass
from pydantic import BaseModel
import torch

@dataclass
class Trajectory:
    episode: Episode
    terminus: Completion
    abs_advantage: float
    token_count: int

    def score(self) -> float:
        return self.episode.weight * self.abs_advantage / self.token_count


def best_trajectory(episode: Episode) -> Trajectory:
    return max(
        (
            Trajectory(
                episode=episode,
                terminus=completion,
                abs_advantage=completion.all_abs_advantage(),
                token_count=completion.all_token_count(tokenizer),
            )
            for completion in episode.completion.leaves()
        ),
        key=lambda t: t.abs_advantage / t.token_count,
    )

In [144]:
2 ** 13

8192

In [147]:
seqlen = 8192
rows = 64

tokens_file = torch.empty(rows * seqlen, dtype=torch.int64)
tokens_file.numpy().tofile("/tmp/tokens.bin")

advantages_file = torch.empty(rows * seqlen, dtype=torch.float32)
advantages_file.numpy().tofile("/tmp/advantages.bin")

logprobs_file = torch.empty(rows * seqlen, dtype=torch.float32)
logprobs_file.numpy().tofile("/tmp/logprobs.bin")

In [157]:
class Request(BaseModel):
    tokens_filename: str
    advantages_filename: str
    logprobs_filename: str
    rows: int
    seqlen: int
    start: int
    stop: int


def handle_request(request: Request) -> None:
    tokens = torch.from_file(
        request.tokens_filename,
        shared=True,
        size=request.rows * request.seqlen,
        dtype=torch.int64,
    ).view(-1, request.seqlen)
    advantages = torch.from_file(
        request.advantages_filename,
        shared=True,
        size=request.rows * request.seqlen,
        dtype=torch.float32,
    ).view(-1, request.seqlen)
    logprobs = torch.from_file(
        request.logprobs_filename,
        shared=True,
        size=request.rows * request.seqlen,
        dtype=torch.float32,
    ).view(-1, request.seqlen)
    trajectories = sorted(
        (best_trajectory(episode) for episode in buffer),
        key=lambda t: t.score(),
    )
    for row in range(request.start, request.stop):
        selected_trajectories: list[Trajectory] = []
        for i in range(0, len(trajectories), -1):
            if (
                trajectories[i].token_count
                + sum(t.token_count for t in selected_trajectories)
                > request.seqlen
            ):
                continue
            selected_trajectories.append(trajectories.pop(i))
        tokens[row] = tokenizer.encode(
            [
                trajectory.terminus.all_message_params()
                for trajectory in selected_trajectories
            ],  # type: ignore
            concatenate=True,
            seqlen=request.seqlen,
        )
        replacement_token = "<|reserved_special_token_250|>"
        mask = tokenizer.encode(
            [trajectory.terminus.all_message_params(replacement_token=replacement_token) for trajectory in selected_trajectories],  # type: ignore
            concatenate=True,
            seqlen=request.seqlen,
        ) == tokenizer.get_token_id(replacement_token)
        mask_size = mask.sum()
        advantages[row] = torch.full_like(mask, fill_value=torch.nan, dtype=torch.float32)
        advantages[row][mask] = torch.tensor(
            list(
                advantage
                for trajectory in selected_trajectories
                for advantage in trajectory.terminus.all_token_advantages()
            )[:mask_size]
        )
        logprobs[row] = torch.full_like(mask, fill_value=torch.nan, dtype=torch.float32)
        logprobs[row][mask] = torch.tensor(
            list(
                advantage
                for trajectory in selected_trajectories
                for advantage in trajectory.terminus.all_logprobs()
            )[:mask_size]
        )

handle_request(
    Request(
        tokens_filename="/tmp/tokens.bin",
        advantages_filename="/tmp/advantages.bin",
        logprobs_filename="/tmp/logprobs.bin",
        rows=64,
        seqlen=8192,
        start=0,
        stop=1,
    )
)

IndexError: list index out of range